## I. Importing libraries and visualizing data and its properties

In [128]:
import numpy as np
import pandas as pd
import re, string, unicodedata
import nltk                                   # Natural language processing tool-kit
# nltk.download('punkt')
# !pip install contractions
# nltk.download('stopwords')
# nltk.download('wordnet')
import contractions


from bs4 import BeautifulSoup               
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet  
from nltk.stem import WordNetLemmatizer

In [144]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('John', pos='n')

'John'

In [129]:
'''importing data as a dataframe'''
data=pd.read_csv('../Data/Tweets.csv')
data_all = pd.read_csv('../Data/Tweets.csv')
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [130]:
'''properties of data'''
print('Shape of dataframe is: ',data.shape)
data.describe()

Shape of dataframe is:  (14640, 15)


,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


## II. Understanding the data

In [131]:
data.drop(data.columns.difference(['airline_sentiment','text']), 1, inplace=True) # Dropping all columns except text and airline sentiment

In [132]:
pd.set_option('display.max_colwidth', None)
print('Shape of the modified dataframe is: ',data.shape)
data.head(10) # Printing first five rows of the data

Shape of the modified dataframe is:  (14640, 2)


,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials to the experience... tacky.
2,neutral,@VirginAmerica I didn't today... Must mean I need to take another trip!
3,negative,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse"
4,negative,@VirginAmerica and it's a really big bad thing about it
5,negative,@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA
6,positive,"@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)"
7,neutral,"@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP"
8,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"
9,positive,"@VirginAmerica it was amazing, and arrived an hour early. You're too good to me."


## III. Pre-processing the data

In [145]:
# a. HTML tag removal
def html_removal(text):
    parsed=BeautifulSoup(text, "html.parser")
    return parsed.get_text()

'''pre-processing before tokenization'''
def contractions_replace(text):
    return contractions.fix(text)

# b. Tokenization: Performed in the for loop

# c. Numbers removal
def rem_nums(text):
    return re.sub(r'\d+', '', text)

# d. Removal of special characters and punctuations
def rem_punct_spchar(words):
    new_words = []
    for word in words:
        new_word =  re.sub(r'[^\w\s]', '', word)
        if new_word!='':
            new_words.append(new_word)
    return new_words

# e. Conversion to lowercase
def to_lowerc(words):
    new_words=[]
    for word in words:
        new_words.append(word.lower())
    return new_words
    
# f. Lemmatize/stemming
def lemm(words):
    lemmatizer = WordNetLemmatizer()
    new_words=[]
    for word in words:
        new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words

# g. Join the words into a string performed in the for loop

In [146]:
words_list=[]; new_words_no_punc=[]; new_words_lowercase=[]; new_words_lemm=[]
lemmatizer = WordNetLemmatizer()
for i  in range(data.shape[0]):
    text = html_removal(data.text[i]) #a.
    text = contractions_replace(text) 
    text = rem_nums(text) #c.
    words_list=nltk.word_tokenize(text) #b.
    new_words_no_punc = rem_punct_spchar(words_list) #d.
    new_words_lowercase = to_lowerc(new_words_no_punc) #e.
    new_words_lemm = lemm(new_words_lowercase) #f.
    data.text[i] = ' '.join(new_words_lowercase) #g.

In [148]:
data.head(5)

,airline_sentiment,text
0,neutral,virginamerica what dhepburn said
1,positive,virginamerica plus you have added commercials to the experience tacky
2,neutral,virginamerica i did not today must mean i need to take another trip
3,negative,virginamerica it is really aggressive to blast obnoxious entertainment in your guests faces they have little recourse
4,negative,virginamerica and it is a really big bad thing about it


In [156]:
'''encoding airline sentiments into integers before vectorization and splitting into training and testing data'''
print('Null values present: True or False? ', data.isnull().values.any())
rstruct={'airline_sentiment':{'neutral':0, 'positive':1, 'negative':2}}
data=data.replace(rstruct)
data.head()

Null values present: True or False?  False


,airline_sentiment,text
0,0,virginamerica what dhepburn said
1,1,virginamerica plus you have added commercials to the experience tacky
2,0,virginamerica i did not today must mean i need to take another trip
3,2,virginamerica it is really aggressive to blast obnoxious entertainment in your guests faces they have little recourse
4,2,virginamerica and it is a really big bad thing about it


## IV. Vectorization